In [69]:
!pip install geopandas

    100% |████████████████████████████████| 931kB 16.5MB/s ta 0:00:01
    100% |████████████████████████████████| 1.8MB 17.2MB/s ta 0:00:01
    100% |████████████████████████████████| 14.7MB 3.3MB/s eta 0:00:01
    100% |████████████████████████████████| 10.4MB 5.1MB/s eta 0:00:01
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [107]:
import pandas as pd
import geopandas as gpd
from glob import glob
import psycopg2

In [6]:
failure_files = glob('../data/Kes*.xlsx')
failure_files                   

['../data/Keskeytyslista_2014.xlsx',
 '../data/Keskeytyslista_2015.xlsx',
 '../data/Keskeytyslista_2013.xlsx',
 '../data/Keskeytyslista_2010.xlsx',
 '../data/Keskeytyslista_2012.xlsx',
 '../data/Keskeytyslista_2011.xlsx',
 '../data/Keskeytyslista_2009.xlsx',
 '../data/Keskeytyslista_2016.xlsx']

In [39]:
col_names = ['identifier', 'start', 'Kesto', 'Lähtö', 'type', 'source', 'desc', 'sijainti', 'rem']
dfs = []
for f in failure_files:
    df = pd.read_excel(f, skiprows=range(0,5), parse_dates=['start'], names=col_names)
    df['duration'] = pd.to_timedelta(df['Kesto'])
    df.loc[:,'end'] = df.loc[:,'start'] + df.loc[:, 'duration']
    df['desc'] = df.desc.fillna('')
    df = df[~df.desc.str.startswith('X14 R')]
    df.drop(columns=['Kesto', 'Lähtö', 'source', 'sijainti', 'rem'], inplace=True)
    dfs.append(df)
outage = pd.concat(dfs)
outage
#pd.to_timedelta(df['Kesto'])

,identifier,start,type,desc,duration,end
0,0-2014-23,2014-01-03 09:39:25,Keskijännitevika,X14 T Tuntematon,00:06:56,2014-01-03 09:46:21
1,0-2014-72,2014-01-07 10:58:09,Keskijännitevika,X14 L1 Tuuli ja myrsky,01:45:16,2014-01-07 12:43:25
2,0-2014-142,2014-01-08 02:18:17,Keskijännitevika,X14 L1 Tuuli ja myrsky,02:57:28,2014-01-08 05:15:45
3,0-2014-147,2014-01-08 09:06:26,Keskijännitevika,X14 L2 Lumi ja jää,00:01:01,2014-01-08 09:07:27
4,0-2014-148,2014-01-08 09:22:48,Keskijännitevika,X14 L1 Tuuli ja myrsky,01:01:57,2014-01-08 10:24:45
6,0-2014-157,2014-01-08 13:29:00,Keskijännitevika,X14 L1 Tuuli ja myrsky,00:09:00,2014-01-08 13:38:00
7,0-2014-169,2014-01-09 09:21:56,Keskijännitevika,X14 L1 Tuuli ja myrsky,00:12:55,2014-01-09 09:34:51
8,0-2014-173,2014-01-09 10:26:00,Keskijännitevika,X14 L1 Tuuli ja myrsky,00:07:00,2014-01-09 10:33:00
9,0-2014-174,2014-01-09 10:50:00,Keskijännitevika,X14 L1 Tuuli ja myrsky,00:10:09,2014-01-09 11:00:09
10,0-2014-189,2014-01-09 12:07:21,Keskijännitevika,X14 L1 Tuuli ja myrsky,00:10:39,2014-01-09 12:18:00


In [40]:
station_files = glob('../data/Vik*.xlsx')
station_files         

['../data/Vikapaikat_2009.xlsx',
 '../data/Vikapaikat_2011.xlsx',
 '../data/Vikapaikat_2012.xlsx',
 '../data/Vikapaikat_2015.xlsx',
 '../data/Vikapaikat_2010.xlsx',
 '../data/Vikapaikat_2014.xlsx',
 '../data/Vikapaikat_2016.xlsx',
 '../data/Vikapaikat_2013.xlsx']

In [51]:
col_names = ['outage_identifier', 'time', 'lng', 'lat', '_', '__', '___', '____', '_____', '.']
dfss = []
for f in station_files:
    df = pd.read_excel(f, skiprows=range(0,6), names=col_names)
    df.drop(columns=['time', '_', '__', '___', '____', '_____', '.'], inplace=True)
    df['code'] = -1
    df['area'] = ''
    df['customers'] = -1
    dfss.append(df)
station = pd.concat(dfss)
station

,outage_identifier,lng,lat,code,area,customers
0,0-2009-3947,606456243,854025595,-1,,-1
1,0-2009-1440,559922846,875577656,-1,,-1
2,0-2009-3951,531506736,854257425,-1,,-1
3,0-2009-862,444854236,872908184,-1,,-1
4,0-2009-3642,464722912,875391736,-1,,-1
5,0-2009-136,521588893,826265797,-1,,-1
6,0-2009-644,437458157,860306535,-1,,-1
7,0-2009-1798,457593932,850913461,-1,,-1
8,0-2009-164,477802980,846482251,-1,,-1
9,0-2009-4556,531535448,888826785,-1,,-1


In [62]:
j = outage.merge(station, left_on='identifier', right_on='outage_identifier')
outage_w_loc = j.loc[:,['identifier', 'start', 'end', 'desc', 'type']]
station_w_loc = j.loc[:, ['outage_identifier', 'lng', 'lat', 'code', 'area', 'customers']]

In [67]:
print(outage_w_loc.head())
print(station_w_loc.head())

  identifier               start                 end                desc  \
0   0-2015-7 2015-01-01 17:44:57 2015-01-01 23:08:08       T1 Tuntematon   
1   0-2015-7 2015-01-01 17:44:57 2015-01-01 23:08:08       T1 Tuntematon   
2  0-2015-23 2015-01-02 23:35:27 2015-01-03 02:42:53  L1 Tuuli ja myrsky   
3  0-2015-23 2015-01-02 23:35:27 2015-01-03 02:42:53  L1 Tuuli ja myrsky   
4  0-2015-26 2015-01-03 00:16:18 2015-01-03 04:30:08  L1 Tuuli ja myrsky   

               type  
0  Keskijännitevika  
1  Keskijännitevika  
2  Keskijännitevika  
3  Keskijännitevika  
4  Keskijännitevika  
  outage_identifier        lng        lat  code area  customers
0          0-2015-7  588372759  838315608    -1              -1
1          0-2015-7  588372759  838315608    -1              -1
2         0-2015-23  482687102  821483099    -1              -1
3         0-2015-23  482687102  821483099    -1              -1
4         0-2015-26  487182290  793813673    -1              -1


In [82]:
def lng(row): 
    lng = 3000000+row.lng/1000
    return lng
def lat(row): 
    lat = 6000000+row.lat/1000
    return lat

station_w_loc['lng'] = station_w_loc.apply(lambda row: lng(row), axis=1)
station_w_loc['lat'] = station_w_loc.apply(lambda row: lat(row), axis=1)
station_w_loc

,outage_identifier,lng,lat,code,area,customers,geometry
0,0-2015-7,3588372.759,6838315.608,-1,,-1,POINT (588372759.000 838315608.000)
1,0-2015-7,3588372.759,6838315.608,-1,,-1,POINT (588372759.000 838315608.000)
2,0-2015-23,3482687.102,6821483.099,-1,,-1,POINT (482687102.000 821483099.000)
3,0-2015-23,3482687.102,6821483.099,-1,,-1,POINT (482687102.000 821483099.000)
4,0-2015-26,3487182.290,6793813.673,-1,,-1,POINT (487182290.000 793813673.000)
5,0-2015-26,3487182.290,6793813.673,-1,,-1,POINT (487182290.000 793813673.000)
6,0-2015-103,3482899.509,6896683.388,-1,,-1,POINT (482899509.000 896683388.000)
7,0-2015-103,3482899.509,6896683.388,-1,,-1,POINT (482899509.000 896683388.000)
8,0-2015-111,3497521.539,6854034.445,-1,,-1,POINT (497521539.000 854034445.000)
9,0-2015-114,3473627.987,6891595.010,-1,,-1,POINT (473627987.000 891595010.000)


In [92]:
station_w_loc_ = gpd.GeoDataFrame(station_w_loc, 
                                  geometry=gpd.points_from_xy(station_w_loc.lng, station_w_loc.lat),
                                  crs="EPSG:2393").to_crs('EPSG:4326')
station_w_loc_

,outage_identifier,lng,lat,code,area,customers,geometry
0,0-2015-7,3588372.759,6838315.608,-1,,-1,POINT (28.66395 61.64220)
1,0-2015-7,3588372.759,6838315.608,-1,,-1,POINT (28.66395 61.64220)
2,0-2015-23,3482687.102,6821483.099,-1,,-1,POINT (26.67170 61.50087)
3,0-2015-23,3482687.102,6821483.099,-1,,-1,POINT (26.67170 61.50087)
4,0-2015-26,3487182.290,6793813.673,-1,,-1,POINT (26.75804 61.25275)
5,0-2015-26,3487182.290,6793813.673,-1,,-1,POINT (26.75804 61.25275)
6,0-2015-103,3482899.509,6896683.388,-1,,-1,POINT (26.66848 62.17565)
7,0-2015-103,3482899.509,6896683.388,-1,,-1,POINT (26.66848 62.17565)
8,0-2015-111,3497521.539,6854034.445,-1,,-1,POINT (26.94979 61.79335)
9,0-2015-114,3473627.987,6891595.010,-1,,-1,POINT (26.49128 62.12946)


In [86]:
!conda install geoplot -c conda-forge
import geoplot

Solving environment: - ^C
failed

CondaError: KeyboardInterrupt



ModuleNotFoundError: No module named 'geoplot'

In [ ]:
finland = world.query('country == "Finland"')
ax = geoplot.cartogram(
    finland, scale='pop_est', limits=(0.2, 1),
    edgecolor='None', figsize=(7, 8)
)
geoplot.polyplot(finland, edgecolor='gray', ax=ax)

In [102]:
station_ = station_w_loc_.loc[:,['outage_identifier','code','area','customers']]
def p_to_lng(row):
    return row.geometry.coords[0][0]
def p_to_lat(row):
    return row.geometry.coords[0][1]

station_['lng'] = station_w_loc_.apply(lambda row: p_to_lng(row), axis=1)
station_['lat'] = station_w_loc_.apply(lambda row: p_to_lat(row), axis=1)
station_

,outage_identifier,code,area,customers,lng,lat
0,0-2015-7,-1,,-1,28.663955,61.642201
1,0-2015-7,-1,,-1,28.663955,61.642201
2,0-2015-23,-1,,-1,26.671700,61.500870
3,0-2015-23,-1,,-1,26.671700,61.500870
4,0-2015-26,-1,,-1,26.758038,61.252750
5,0-2015-26,-1,,-1,26.758038,61.252750
6,0-2015-103,-1,,-1,26.668479,62.175655
7,0-2015-103,-1,,-1,26.668479,62.175655
8,0-2015-111,-1,,-1,26.949789,61.793354
9,0-2015-114,-1,,-1,26.491284,62.129459


We need to load old data to remove duplicate entries

In [116]:
def load_dataset(table_name='classification_dataset', cols=[]):
    """
    Load classification dataset from db
    
    table_name : str
                 table name to be used
    """

    conn = psycopg2.connect("dbname='%s' user='%s' host='%s' password='%s'" % (db_name, db_user, db_host, db_pass))
    sql = """SELECT "{}" FROM sasse.{}""".format('","'.join(cols), table_name)

    cursor = conn.cursor()
    cursor.execute(sql)
    results = cursor.fetchall()
    df = pd.DataFrame(results, columns=cols)

    return df 

old_station = load_dataset(table_name='station', cols=['outage_identifier'])
old_outage = load_dataset(table_name='outage', cols=['identifier'])

In [132]:
outage_to_save = outage_w_loc[~outage_w_loc.identifier.isin(old_outage.loc[:,'identifier'])]
station_to_save = station_[~station_.outage_identifier.isin(old_station.loc[:,'outage_identifier'])]
outage_to_save.drop_duplicates(subset=['identifier'], inplace=True)
station_to_save.drop_duplicates(subset=['outage_identifier'], inplace=True)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [133]:
print(old_outage[old_outage.loc[:,'identifier'] == '0-2015-7'])
print(outage_to_save[outage_to_save.loc[:,'identifier'] == '0-2015-7'])

Empty DataFrame
Columns: [identifier]
Index: []
  identifier               start                 end           desc  \
0   0-2015-7 2015-01-01 17:44:57 2015-01-01 23:08:08  T1 Tuntematon   

               type  
0  Keskijännitevika  


In [134]:
from sqlalchemy import create_engine
db_host = '***REMOVED***'
db_port = 5432
db_name = "postgres"
db_user = "***REMOVED***"
db_pass = "***REMOVED***"
def save_dataset(df, table_name='outage'):
    """
    Save dataset into the db

    df : DataFrame
         DataFrame data
    """
    if df is None or len(df) < 1:
        return
    print('Storing classification set to db sasse.{}...'.format(table_name))

    # db_name, db_user, db_host, db_pass
    engine = create_engine('postgresql://{user}:{passwd}@{host}:5432/{db}'.format(user=db_user,
                                                                                 passwd=db_pass,
                                                                                 host=db_host,
                                                                                 db=db_name))

    df.to_sql(table_name, engine, schema='sasse', if_exists='append', index=False)

save_dataset(outage_to_save, table_name='outage')
save_dataset(station_to_save, table_name='station')

Storing classification set to db sasse.outage...
Storing classification set to db sasse.station...
